In [2]:
import os
os.chdir('..')
!pwd

/home/oguz/Workspace/smart_evidence


In [3]:
import srsly
from tqdm.notebook import tqdm
import pickle


In [ ]:
import spacy
nlp = spacy.load('en_core_web_trf')

In [ ]:
items = srsly.read_jsonl("corpus/paragraphs.jsonl")

ncs = []

for i, doc in tqdm(enumerate(nlp.pipe([item['text'] for item in items]))):
    ncs_for_par = [tok for tok in [' '.join([tok.lemma_.lower() for tok in nc if tok.pos_ not in  ['DET', 'PRON', "PUNCT"]]) for nc in doc.noun_chunks] if tok]
    ncs.append(ncs_for_par)
    if (i+1) % 100:
        with open('nounchunks2.pkl', 'wb') as f:
            pickle.dump(ncs, f)


0it [00:00, ?it/s]

KeyboardInterrupt: 

In [ ]:
with open('nounchunks.pkl', 'rb') as f:
    ncs = pickle.load(f)

In [6]:
import re
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer=lambda x: [t for t in x if re.match(r'^[a-zA-Z].*[a-zA-Z\d]$', t)])
X = vectorizer.fit_transform(ncs)

In [7]:
import numpy as np

In [8]:
np.array(vectorizer.get_feature_names())[np.array(X.sum(0))[0].argsort()[-10:]]

array(['case', 'gas', 'electricity', 'cost', 'year', 'development', 'use',
       'government', 'example', 'country'], dtype='<U379')

In [9]:
from sklearn.preprocessing import normalize

In [10]:
A = normalize(X, axis=1)

In [14]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=1000, n_iter=7, random_state=42)

In [15]:
A = svd.fit_transform(A)

In [23]:
from scipy.sparse import csr_matrix, csc_matrix

In [24]:
S = csr_matrix(A)

In [25]:
C = S.dot(S.T)

In [1]:
C.shape                                                                                                 

NameError: name 'C' is not defined

In [19]:
vocab_index = vectorizer.get_feature_names().index('recycling')
np.array(vectorizer.get_feature_names())[C[vocab_index].toarray()[0].argsort()[-20:]]

IndexError: index 361586 is out of bounds for axis 0 with size 1000